# Paso 1.-  Datos del precio de venta cliente

Como primer paso vamos a obtener los datos de precio venta cleinte de la web https://www.esios.ree.es/. Esta web nos proporciona por medio de una Api multitud de datos que podemos utilizar. Para la correcta extracción vamos a utilizar algunas librerías.

## Librerías:

Utilizaremos la Librería de https://pypi.org/project/esiosdata/ que permite acceder de manera fácil a la API. Instalación: 
- Se debe instalar por medio de "conda install pip"
- https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-pkgs.html#installing-non-conda-packages
- pip3 install esiosdata (dentro del env) (pip simple si no funciona)
- echo "874b2aeaf4071f8db7085e0ca6aa88ede7e2992360fc0d6ad7af625b30b9e3bf" > ~/.token_api_esios



## Obtención de datos PVPC
Vamos a obtener los datos de PVPC y de sus tarifas relacionadas. 

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from esiosdata import PVPC
from esiosdata.prettyprinting import *
from datetime import datetime

pvpc_handler = PVPC()

--> NO SE LEE DB_HDF (Exception: ValueError:The file '/Users/davidcerezallanda/.esiosdata/esiospvpc.h5' is already opened, but not in read-only mode (as requested).)
--> Se procede a realizar la captura de TODOS los datos existentes:

PROCESADO DE DATOS WEB DE 1958 DÍAS
Procesado de tareas en paralelo [1->50, %=2.6]: 1.75 seg [0.0350 seg/tarea]
Procesado de tareas en paralelo [51->100, %=5.1]: 1.63 seg [0.0327 seg/tarea]
Procesado de tareas en paralelo [101->150, %=7.7]: 1.62 seg [0.0324 seg/tarea]
Procesado de tareas en paralelo [151->200, %=10.2]: 1.61 seg [0.0322 seg/tarea]
Fichero irregular nº horas != 24: --> 26-10-2014 -> 25 medidas
Procesado de tareas en paralelo [201->250, %=12.8]: 1.68 seg [0.0336 seg/tarea]
Procesado de tareas en paralelo [251->300, %=15.3]: 1.63 seg [0.0326 seg/tarea]
Procesado de tareas en paralelo [301->350, %=17.9]: 1.61 seg [0.0322 seg/tarea]
Fichero irregular nº horas != 24: --> 29-03-2015 -> 23 medidas
Procesado de tareas en paralelo [351->400, %=20.4]

In [10]:
import pandas as pd
import numpy as np

df_pvpc = pvpc_handler.data['data']
df_pvpc.tail()

,CCVGEN,CCVNOC,CCVVHC,COFGEN,COFNOC,COFVHC,FOMGEN,FOMNOC,FOMVHC,FOSGEN,FOSNOC,FOSVHC,...,PMHGEN,PMHNOC,PMHVHC,SAHGEN,SAHNOC,SAHVHC,TEUGEN,TEUNOC,TEUVHC,VHC,date,date_timestamp
fecha,,,,,,,,,,,,,,,,,,,,,,,,,
2019-08-10 19:00:00+02:00,1.85,1.87,1.87,0.000125,0.000076,0.000062,0.03,0.03,0.03,0.13,0.13,0.13,...,52.46,52.86,52.86,1.22,1.23,1.23,44.03,62.01,62.01,124.58,2019-08-10 19:00:00+02:00,1.565456e+09
2019-08-10 20:00:00+02:00,1.93,1.95,1.95,0.000128,0.000081,0.000064,0.03,0.03,0.03,0.13,0.13,0.13,...,57.17,57.61,57.61,1.15,1.16,1.16,44.03,62.01,62.01,129.37,2019-08-10 20:00:00+02:00,1.565460e+09
2019-08-10 21:00:00+02:00,1.97,1.98,1.98,0.000139,0.000128,0.000146,0.03,0.03,0.03,0.13,0.13,0.13,...,59.53,60.04,60.04,1.07,1.08,1.08,44.03,62.01,62.01,131.77,2019-08-10 21:00:00+02:00,1.565464e+09
2019-08-10 22:00:00+02:00,1.96,1.98,1.98,0.000136,0.000150,0.000199,0.03,0.03,0.03,0.13,0.13,0.13,...,58.74,59.24,59.24,1.17,1.18,1.18,44.03,62.01,62.01,131.07,2019-08-10 22:00:00+02:00,1.565467e+09
2019-08-10 23:00:00+02:00,1.93,1.80,1.88,0.000122,0.000147,0.000184,0.03,0.03,0.03,0.13,0.13,0.13,...,56.15,54.11,56.38,1.39,1.33,1.39,44.03,2.22,2.88,64.87,2019-08-10 23:00:00+02:00,1.565471e+09


#### Limpiar cierta información no usable

Ponemos como límite inferior el día 1 de Octubre del 2015, será el que utilizaremos en los demás parámetros ya que es el primer día del que se dispone del preci PVPC. 

In [11]:
df_pvpc = df_pvpc[df_pvpc['date'] >= '2015-10-01']

#Other prices cleanup
df_pvpc = df_pvpc[pvpc_handler.tarifas]
df_pvpc.rename({'GEN': 'PVPC_DEF', 'NOC': 'PVPC_2_PED_NOC', 'VHC': 'PVPC_ELEC_NOC'}, axis=1, inplace=True)

#timestamp adding
df_pvpc['date'] = pd.to_datetime(df_pvpc.index)
df_pvpc['date_timestamp'] = df_pvpc['date'].apply(lambda x:  datetime.timestamp(x))
print(df_pvpc.shape)

(33840, 5)


In [12]:
df_pvpc.head()

,PVPC_DEF,PVPC_2_PED_NOC,PVPC_ELEC_NOC,date,date_timestamp
fecha,,,,,
2015-10-01 00:00:00+02:00,117.77,69.49,71.88,2015-10-01 00:00:00+02:00,1.443650e+09
2015-10-01 01:00:00+02:00,117.09,68.21,65.57,2015-10-01 01:00:00+02:00,1.443654e+09
2015-10-01 02:00:00+02:00,114.59,65.34,62.53,2015-10-01 02:00:00+02:00,1.443658e+09
2015-10-01 03:00:00+02:00,116.60,66.53,63.40,2015-10-01 03:00:00+02:00,1.443661e+09
2015-10-01 04:00:00+02:00,122.07,71.03,67.37,2015-10-01 04:00:00+02:00,1.443665e+09


In [13]:
df_pvpc.to_csv('data_precios.csv')